In [8]:

import numpy as np
import prody
from potsim2 import PotGrid
import os
import shutil
from openbabel import openbabel
import subprocess
import jinja2

In [21]:

pdb2pqrPath = 'pdb2pqr'
SRC_DIR = '/Users/tevfik/Sandbox/github/PHD/data/src_data/pdbbind/refined-set'
DEST_DIR = '/Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step'
APBS_TEMPLATE_DIR = '/Users/tevfik/Sandbox/github/PHD/3dunet-cavity'
PROTEIN_NAME = '2dw7'

srcPdbFile = f"{SRC_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_protein.pdb"
srcMolFile = f"{SRC_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_protein.mol2"
destPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}.pdb"
destLigandPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_ligand.pdb"
destSelectedPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_selected.pdb"
destCalculatedPocketFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_pocket.pdb"
destSelectedGridFile  = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_grid.dx"
destSelectedPqrFile  = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}.pqr"
destNumpyGridFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_grid.npy"

# Convert Protein to pdb.
copy_path = DEST_DIR +"/" + PROTEIN_NAME 
if not os.path.exists(copy_path):
    os.makedirs(copy_path)
if os.path.exists(srcPdbFile):
    print(f"Found existing protein pdb file in {srcPdbFile}. Copying over to {destPdbFile}")
    shutil.copyfile(srcPdbFile, destPdbFile)
elif os.path.exists(srcMolFile):
    print(f"Found existing protein MOL file in {srcMolFile}. Converting to {destPdbFile}")
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("mol2", "pdb")
    # convert protein to pdb format from mol2 format
    protein = openbabel.OBMol()
    obConversion.ReadFile(protein, srcMolFile)
    obConversion.WriteFile(protein, destPdbFile)

Found existing protein pdb file in /Users/tevfik/Sandbox/github/PHD/data/src_data/pdbbind/refined-set/2dw7/2dw7_protein.pdb. Copying over to /Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step/2dw7/2dw7.pdb


In [14]:
# Using `prody` library remove water molecules from pdb file content removes everything but protein
structure = prody.parsePDB(destPdbFile)
structure = structure.select('protein')
prody.writePDB(destPdbFile, structure)

# pdb2pqr fails to read pdbs with the one line header generated by ProDy...
with open(destPdbFile, 'r') as fin:
    data = fin.read().splitlines(True)
with open(destPdbFile, 'w') as fout:
    fout.writelines(data[1:])

@> 11986 atoms and 1 coordinate set(s) were parsed in 0.28s.


In [17]:
# convert ligand to pdb
ligand = openbabel.OBMol()
srcMolFile1 = f"{SRC_DIR}/{PROTEIN_NAME}/ligand.mol2"
srcMolFile2 = f"{SRC_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_ligand.mol2"
destLigandPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_ligand.pdb"

obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("mol2", "pdb")

if os.path.exists(srcMolFile1):
    obConversion.ReadFile(ligand, srcMolFile1)
elif os.path.exists(srcMolFile2):
    obConversion.ReadFile(ligand, srcMolFile2)
else:
    print("No ligand mol file found. Exiting.", file=sys.stderr)
    raise Exception("No input files found")
obConversion.WriteFile(ligand, destLigandPdbFile)



True

In [18]:
# Select only chanins that are close to ligand
protein = prody.parsePDB(destPdbFile)
ligand = prody.parsePDB(destLigandPdbFile)
lresname = ligand.getResnames()[0]
complx = ligand + protein
complx = complx.select(f'same chain as exwithin 7 of resname {lresname}')

# Select only atoms that belongs to the protein
structure = complx.select(f'protein and not resname {lresname}')
prody.writePDB(destSelectedPdbFile, structure)
# pdb2pqr fails to read pdbs with the one line header generated by ProDy...
with open(destSelectedPdbFile, 'r') as fin:
    data = fin.read().splitlines(True)
with open(destSelectedPdbFile, 'w') as fout:
    fout.writelines(data[1:])

selected = prody.parsePDB(destSelectedPdbFile)   



@> 11986 atoms and 1 coordinate set(s) were parsed in 0.14s.
@> 14 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 11986 atoms and 1 coordinate set(s) were parsed in 0.32s.


In [19]:
# Calculate packet and write it to corresponding pocket file.

selected = prody.parsePDB(destSelectedPdbFile)
complx = ligand + selected
lresname = ligand.getResnames()[0]
pocket = complx.select(f'same residue as exwithin 4.5 of resname {lresname}')
prody.writePDB(destCalculatedPocketFile, pocket)  



@> 11986 atoms and 1 coordinate set(s) were parsed in 0.13s.


'/Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step/2dw7/2dw7_pocket.pdb'

In [22]:
# Run pdbqr for selected protein
# PDB2PQR:
#   This task Fixes PDB, adds charges and radius. In order to perform electrostatics calculations on your biomolecular structure of interest, 
#   you need to provide atomic charge and radius information to APBS. Charges are used to form the biomolecular charge distribution 
#   for the Poisson-Boltzmann (PB) equation while the radii are used to construct the dielectric and ionic accessibility functions.
print(os.getcwd())
print(destSelectedPdbFile)
print(destSelectedPqrFile)
proc = subprocess.Popen(
    [
        pdb2pqrPath,
        "--with-ph=7.4",
        "--ff=PARSE",
        "--keep-chain",
        destSelectedPdbFile,
        destSelectedPqrFile,
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
cmd_out = proc.communicate()
if proc.returncode != 0:
    raise Exception(cmd_out[1].decode())

/Users/tevfik/Sandbox/github/PHD/3dunet-cavity/notebooks
/Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step/2dw7/2dw7_selected.pdb
/Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step/2dw7/2dw7.pqr


Exception: INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step/2dw7/2dw7_selected.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 776 residues and 11986 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:This biomolecule is clean.  No repair needed.
INFO:Updating disulfide bridges.
INFO:Debumping biomolecule.
CRITICAL:Unable to debump biomolecule. Biomolecular structure is incomplete:  Found gap in bi

In [43]:
context = {"name": PROTEIN_NAME}
tmpl_env = jinja2.Environment(loader=jinja2.FileSystemLoader(APBS_TEMPLATE_DIR))
str_file = tmpl_env.get_template("apbs-energy.in.tmpl").render(context)
apbs_in_fname = "apbs-in"


current_dir = os.getcwd()
destDir = f"{DEST_DIR}/{PROTEIN_NAME}"
os.chdir(destDir)

with open(apbs_in_fname, "w") as f:
    f.write(str_file)

print(os.getcwd())
proc = subprocess.Popen(
    ["apbs", apbs_in_fname], stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
cmd_out = proc.communicate()
if proc.returncode != 0:
    raise Exception(cmd_out[1].decode())    



import gzip

print(destSelectedGridFile)
with open(destSelectedGridFile, 'rb') as orig_file:
    with gzip.open(f"{destSelectedGridFile}.gz", 'wb') as zipped_file:
        zipped_file.writelines(orig_file)


os.chdir(current_dir)


/Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step/10gs
/Users/tevfik/Sandbox/github/PHD/data/dest_data_step_by_step/10gs/10gs_grid.dx


In [44]:
# Calculate Electrostatic Grid:
# Calculates an electrostatic grid. 
#    It creates a `/dest/{prot}/{prot}_grid.dx.gz`. Saves it to `/dest/{prot}/{prot}_grid.npy`. 
#    This file will be used for training the UNet.

#grid = PotGrid(f"{out_dir}/{self.name}_selected.pdb", f"{out_dir}/{self.name}_grid.dx.gz")
grid = PotGrid(destSelectedPdbFile, destSelectedGridFile)
    # save the grid as .npy file to use it for training
np.save(destNumpyGridFile, grid.grid)

